# Linear Regression in Pytorch 

In [47]:
import numpy as np
import torch

In [48]:
#Input(temp,rainfall,humidity)
inputs=np.array([[73,67,43],
                 [91,88,64],
                 [87,134,58],
                 [102,43,37],
                 [69,96,70]],dtype='float32')

In [49]:
#Targets(apples,oranges)
targets=np.array([[56,70],
                  [81,101],
                  [119,133],
                  [22,37],
                  [103,119]],dtype='float32')

In [50]:
#Convert numpy to Tensor
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Linear Regression model

In [51]:
#Weights and Biases -->craete a tensor randomly with normal distribution with mean 0 and std 1

#yeild apple=w11*temp+w12*rainfall+w13*humidity+b1

#yeild orange=w21*temp+w22*rainfall+w23*humidity+b2

# X(5*3) * WT(3*2) + b(1*2)=Result(5*2)

In [52]:
w=torch.randn(2,3,requires_grad=True)
b=torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[-1.6980, -0.0553,  1.8896],
        [-1.4282,  0.1571,  0.0392]], requires_grad=True)
tensor([-1.4816, -0.1588], requires_grad=True)


In [53]:
#Add Character @-->represents multiplication in PyTorch
#.t transpose of a tensor

In [54]:
w.t()

tensor([[-1.6980, -1.4282],
        [-0.0553,  0.1571],
        [ 1.8896,  0.0392]], grad_fn=<TBackward0>)

In [55]:
inputs@w.t()

tensor([[ -46.4076,  -92.0416],
        [ -38.4521, -113.6253],
        [ -45.5419, -100.9197],
        [-105.6590, -137.4646],
        [   9.7984,  -80.7135]], grad_fn=<MmBackward0>)

In [56]:
def model(x):
    return x@w.t()+b

In [57]:
inputs@w.t()+b

tensor([[ -47.8892,  -92.2004],
        [ -39.9337, -113.7841],
        [ -47.0235, -101.0785],
        [-107.1405, -137.6234],
        [   8.3168,  -80.8724]], grad_fn=<AddBackward0>)

In [58]:
preds=model(inputs)
preds

tensor([[ -47.8892,  -92.2004],
        [ -39.9337, -113.7841],
        [ -47.0235, -101.0785],
        [-107.1405, -137.6234],
        [   8.3168,  -80.8724]], grad_fn=<AddBackward0>)

In [59]:
#compare Targets and Predictions

In [60]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [61]:
#we can see a big difference between our model's predication and the actual targets because w have initialized our model with
# Random weights and Biases.Obviously, we can not expext a random initialized model just work.

In [62]:
preds-targets

tensor([[-103.8892, -162.2004],
        [-120.9337, -214.7841],
        [-166.0235, -234.0785],
        [-129.1405, -174.6234],
        [ -94.6832, -199.8724]], grad_fn=<SubBackward0>)

In [63]:
diff=preds-targets
diff*diff

tensor([[10792.9580, 26308.9727],
        [14624.9492, 46132.2188],
        [27563.7969, 54792.7539],
        [16677.2773, 30493.3477],
        [ 8964.9023, 39948.9648]], grad_fn=<MulBackward0>)

In [64]:
#10 is the number of the elements--->we can use the numel method

In [65]:
diff.numel()

10

In [66]:
torch.sum(diff*diff)/10

tensor(27630.0156, grad_fn=<DivBackward0>)

Loss Function

In [67]:
#MSE loss
def mse(t1,t2):
    diff=t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [68]:
#Numel() method returns the number of a elemnets in a tensor.

In [69]:
#Compute loss

In [70]:
loss=mse(preds,targets)
print(loss)

tensor(27630.0156, grad_fn=<DivBackward0>)


Compute Gradients

With PyTorch we can automatically compute the gradient or derivative of the loss.

In [71]:
loss.backward()

In [72]:
#Gradients for weigths

In [73]:
print(w)
print(w.grad)

tensor([[-1.6980, -0.0553,  1.8896],
        [-1.4282,  0.1571,  0.0392]], requires_grad=True)
tensor([[-10547.6777, -10898.5020,  -6648.4746],
        [-16670.7207, -17566.3008, -10949.8984]])


In [74]:
#w.grad-->represents the derivatives of loss with the respect of specific weights elements

In [75]:
print(b)
print(b.grad)

tensor([-1.4816, -0.1588], requires_grad=True)
tensor([-122.9340, -197.1118])


Adjust weights and Biases using Gradients to reduce the loss

In [76]:
with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5

In [77]:
w,b

(tensor([[-1.5925,  0.0537,  1.9560],
         [-1.2614,  0.3328,  0.1487]], requires_grad=True),
 tensor([-1.4803, -0.1569], requires_grad=True))

In [78]:
preds=model(inputs)
preds

tensor([[ -30.0273,  -63.5509],
        [ -16.4884,  -76.1455],
        [ -19.3857,  -56.6832],
        [ -89.2344, -109.0124],
        [  30.7124,  -44.8390]], grad_fn=<AddBackward0>)

In [79]:
loss=mse(preds,targets)
print(loss)

tensor(18701.2812, grad_fn=<DivBackward0>)


In [80]:
#we reset the gradients to zero by .zero() method. we need to do this because Pytorch accumulates gradients.

In [81]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Train the model Using Gradients Descent

In [82]:
#Train for 100 epoch
for i in range(100):
    preds=model(inputs)
    loss=mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w-=w.grad*1e-5
        b-=b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

In [83]:
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(147.4864, grad_fn=<DivBackward0>)


In [84]:
#the loss is now much lower

In [85]:
import math

In [86]:
math.sqrt(loss)

12.144398695601787

In [89]:
preds

tensor([[ 53.6453,  69.0065],
        [ 91.2998,  97.5353],
        [103.6670, 142.1817],
        [  3.2136,  29.3606],
        [127.8752, 118.1222]], grad_fn=<AddBackward0>)

In [90]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [43]:
#we can see that predctions is closer to the targets